In [1]:
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta
import time
#import pymc3
import math
import arviz as az
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import seaborn as sns

from zipfile import ZipFile
import scipy as sp


import sys, subprocess, glob, os, shutil, re, importlib
from subprocess import call
import imp


%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import matplotlib.colors as clr
from matplotlib import rc
import textwrap as textwrap
from textwrap import wrap


from scipy.special import binom

In [2]:
log_file_path = "/Users/mparedes/Desktop/mpox_rhino/300_region_updated_skyline.log"


In [3]:
burnin_percent = 0.30

In [4]:
def read_in_Ne_changes_mascot(log_file_path):
    
    Ne_skyline_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                #print(line)
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "Ne." in col:
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        deme = name.split(".")[1]# the syntax here is "Ne.region.1" where region is deme and 1 is interval 1
                        interval = name.split(".")[2]
                       
                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])
                    
                
    return(Ne_skyline_dict)

In [5]:
Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

In [6]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "SkylineNe" in i:
            deme = i.split(".")[1]
            #print(deme)
            interval = i.split(".")[2]
            #print(interval)
            #print(interval)
            #print(i)
            next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = 10**mean_log
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
            
            try:
                next_local_series = input_df["SkylineNe"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
                diff_series = np.subtract(local_series, next_local_series)
                #print(local_series)
                #print(next_local_series)
                #print(diff_series)
                diff_mean_log = diff_series.mean()
                diff_median_log = np.median(diff_series)
                diff_hpd_95 = az.hdi(diff_series, 0.95)
                diff_lower_hpd_log_95 = diff_hpd_95[0]
                diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
                diff_upper_hpd_log_95 = diff_hpd_95[1]
                diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
                diff_hpd_50 = az.hdi(diff_series, 0.50)
                diff_lower_hpd_log_50 = diff_hpd_50[0]
                diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
                diff_upper_hpd_log_50 = diff_hpd_50[1]
                diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
            except KeyError:
                pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"deme":deme, "interval":interval, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "median_Ne_log" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                   "diff_mean_Ne_log":diff_mean_log, "diff_mean_log":diff_mean_log,
                                                   "diff_upper_hpd_log_95":diff_upper_hpd_log_95,"diff_lower_hpd_log_95":diff_lower_hpd_log_95, 
                                                   "diff_upper_hpd_log_50":diff_upper_hpd_log_50,"diff_lower_hpd_log_50":diff_lower_hpd_log_50,
                                                   "diff_upper_hpd_linear":diff_upper_hpd_linear_95,"diff_lower_hpd_linear":diff_lower_hpd_linear_95,
                                                   "diff_upper_hpd_linear_50":diff_upper_hpd_linear_50,"diff_lower_hpd_linear_50":diff_lower_hpd_linear_50})
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
    return(new_df)

In [7]:
Ne_df = pd.DataFrame.from_dict(Ne_skyline)
print(len(Ne_df))
Ne_df

40509


,sample,SkylineNe.CentralEurope.1,SkylineNe.CentralEurope.2,SkylineNe.CentralEurope.3,SkylineNe.CentralEurope.4,SkylineNe.CentralEurope.5,SkylineNe.CentralEurope.6,SkylineNe.CentralEurope.7,SkylineNe.CentralEurope.8,SkylineNe.CentralEurope.9,...,SkylineNe.WesternEurope.2,SkylineNe.WesternEurope.3,SkylineNe.WesternEurope.4,SkylineNe.WesternEurope.5,SkylineNe.WesternEurope.6,SkylineNe.WesternEurope.7,SkylineNe.WesternEurope.8,SkylineNe.WesternEurope.9,SkylineNe.WesternEurope.10,SkylineNe.WesternEurope.11\n
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n
1,1000,0.10907159157724362,0.937061001446744,-0.36822676743696764,0.6377191083682311,1.0649902980361703,-0.7607644850909145,-0.3987028862929026,-0.7476790181417562,0.48437130213016955,...,-0.08538475753913224,-0.8604302803386132,-0.7267490367167464,0.21426592701034136,-1.4300836064384477,0.8653250483369149,0.39531741062634984,-1.5526957414545677,0.6030287775584333,-0.62589961302214\n
2,2000,1.0184564391502704,0.7540381061278125,0.11840759885786006,0.49526177696195206,1.1969547510530734,-0.6175860379966205,-0.2639750207140214,-0.7275862635058624,-0.017718470351097623,...,0.24069526789420792,-1.3014392977864984,-1.1238443625682988,1.1548478806046105,-1.3784150533926094,0.7241877776179761,0.911762205261385,-1.8672840084530422,0.6946886445453182,-0.02676348141115878\n
3,3000,0.3145432367167688,-0.8046656860279442,1.5309334652715079,3.9446399297014714,0.4199057559932893,-0.01908664564588672,-0.9544962494029606,0.08805929769980143,1.1108014774970065,...,0.2727901541601254,1.3832577962360686,0.41778360282972316,-0.27920233009606765,-0.42358014146657685,-1.406898610575779,-0.26671442089351516,-0.7662644953854858,-0.34854576899716877,0.2434019795640711\n
4,4000,0.2958372227754861,-0.21682221080859942,2.2093132333279493,4.10415274873988,0.5186024467387257,0.46493896281494207,-2.0013385189631,-0.7433541754168079,0.9439272101991396,...,0.1054481623267984,0.9221111248320366,0.3211458050074005,0.12311225222707546,-0.20312323725309828,-1.6539449511155255,-0.09310765336835414,-0.5574430577535477,0.17311148106796959,0.8676952348136744\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40504,40504000,-0.8611654481965916,-0.5428445170183835,-1.3944545941182598,-2.660123908834681,-3.4281558777852474,-5.195169482690476,-4.939729788768945,-5.460401824670958,-6.8047761089819305,...,-1.2177809707331804,-2.266868530871618,-2.5818021857144124,-1.9314056691627903,-1.4455370977695392,-3.7857525632354743,-3.3823748725379303,-3.720332046991162,-2.800736019230791,-3.5411271964557596\n
40505,40505000,-0.2483928546125871,-0.3891479070740457,-1.7941680700580618,-2.748848420166448,-3.5674510309814575,-5.358162810207602,-5.650811761459721,-5.87462249852079,-7.214207373101933,...,-1.3616855162494135,-2.392946660105191,-2.9820316037050163,-2.400335348440589,-1.5855155652594586,-3.834600480099803,-4.149175631618303,-4.236525722672037,-3.1936765392322335,-3.834766943666944\n
40506,40506000,-0.9240085221843081,-0.9333029842549218,-2.0724814661188957,-2.9708853675748133,-3.548268501782553,-5.13047980478956,-5.458464290391855,-5.920212571065092,-7.377701496889273,...,-1.2182013386938522,-2.22289982617904,-2.912177205896283,-2.4037000458797335,-1.4696757239132399,-3.8967880446280057,-3.8782075113499523,-4.174303892154586,-3.637971372496728,-4.029578476616936\n
40507,40507000,-0.6368926077127908,-1.5116705257823375,-2.131214339951197,-2.6696030387426357,-3.43733019472045,-5.109379099764123,-4.960836387617485,-5.764276090192243,-7.585687961415564,...,-1.5132102105491212,-2.400599453205314,-2.5953850918800656,-1.902994458527226,-1.496056793579145,-3.7284670579843886,-3.0176281193800714,-3.283430190003492,-3.096277735714318,-3.943542348520256\n


In [8]:
ne_summary = generate_summary_df(Ne_df)
ne_summary

/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_3671/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_3671/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_3671/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_3671/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var

/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_3671/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_3671/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_3671/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_3671/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var

,deme,interval,mean_Ne_log,mean_Ne_linear,median_Ne_log,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,upper_hpd_linear,...,diff_mean_Ne_log,diff_mean_log,diff_upper_hpd_log_95,diff_lower_hpd_log_95,diff_upper_hpd_log_50,diff_lower_hpd_log_50,diff_upper_hpd_linear,diff_lower_hpd_linear,diff_upper_hpd_linear_50,diff_lower_hpd_linear_50
0,CentralEurope,1,-0.617314,0.241371,-0.618503,1.162328,-2.410500,0.016494,-1.189226,3.197369,...,0.564960,0.564960,2.357636,-1.222443,1.188026,-0.064341,10.565947,0.294510,3.280599,0.937685
0,CentralEurope,2,-1.182274,0.065724,-1.187068,1.063838,-3.466641,-0.380250,-1.962774,2.897470,...,0.559064,0.559064,2.326467,-1.310337,1.142556,-0.096147,10.241695,0.269729,3.134769,0.908330
0,CentralEurope,3,-1.741338,0.018141,-1.752717,0.691260,-4.280284,-0.980592,-2.739569,1.996230,...,0.494959,0.494959,2.369983,-1.352025,1.129836,-0.125388,10.697214,0.258716,3.095149,0.882154
0,CentralEurope,4,-2.236297,0.005804,-2.329063,0.543432,-4.805463,-1.509685,-3.261762,1.721907,...,0.370684,0.370684,2.136090,-1.524640,0.986772,-0.298084,8.466272,0.217699,2.682561,0.742239
0,CentralEurope,5,-2.606981,0.002472,-2.785397,0.706658,-4.923494,-2.206881,-3.757555,2.027205,...,0.777465,0.777465,2.644694,-1.067379,1.427677,0.164345,14.079140,0.343909,4.169002,1.178620
0,CentralEurope,6,-3.384445,0.000413,-3.587556,-0.329526,-5.803000,-3.141313,-4.593912,0.719265,...,0.857978,0.857978,2.768845,-1.034263,1.548150,0.270175,15.940216,0.355488,4.702764,1.310193
0,CentralEurope,7,-4.242424,0.000057,-4.444224,-1.356520,-6.256000,-4.023513,-5.174899,0.257556,...,0.241329,0.241329,2.267603,-1.767733,0.916643,-0.454251,9.656224,0.170720,2.500881,0.634923
0,CentralEurope,8,-4.483753,0.000033,-4.569044,-1.771411,-7.264036,-3.792469,-5.418729,0.170093,...,0.064664,0.064664,1.947350,-1.823822,0.721334,-0.603373,7.010089,0.161408,2.057176,0.546964
0,CentralEurope,9,-4.548416,0.000028,-4.592659,-1.444774,-8.036932,-3.479239,-5.497158,0.235799,...,0.005065,0.005065,2.018905,-1.941367,0.812165,-0.533639,7.530071,0.143508,2.252780,0.586467
0,CentralEurope,10,-4.553481,0.000028,-4.580554,-0.593865,-8.671930,-3.371712,-5.748727,0.552189,...,-0.020876,-0.020876,1.970560,-1.993818,0.577414,-0.802723,7.174692,0.136175,1.781427,0.448107


In [9]:
ne_summary.lower_hpd_linear_50

0     0.304457
0     0.140468
0     0.064598
0     0.038321
0     0.023341
0     0.010113
0     0.005657
0     0.004433
0     0.004098
0     0.003187
0     0.002398
0     1.162280
0     2.024400
0     1.527746
0     1.137737
0     1.434058
0     1.547795
0     0.566130
0     0.136987
0     0.038092
0     0.020270
0     0.016335
0     0.704247
0     0.972677
0     1.669742
0     1.505436
0     0.410222
0     0.105134
0     0.037845
0     0.022419
0     0.012551
0     0.010867
0     0.008111
0     0.772479
0     1.147879
0     2.019981
0     4.897926
0    18.233433
0    16.775224
0    10.837094
0     0.391775
0     0.085217
0     0.029576
0     0.021025
0     0.395414
0     0.215729
0     0.149012
0     0.149768
0     0.198305
0     0.194118
0     0.127774
0     0.029571
0     0.013201
0     0.007182
0     0.005333
Name: lower_hpd_linear_50, dtype: float64

In [10]:
test = ne_summary
test['days'] = (test.interval.astype(int))*25
test['date'] = dt.strptime("2023-01-03",  "%Y-%m-%d") - test.days.map(timedelta)
#test.date = test.date.astype(str)

In [12]:
domain = ['CentralEurope', 'NorthAmerica', 'SouthAmerica' ,"SouthernEurope", "WesternEurope"]
range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]
line = alt.Chart(test).mark_area(interpolate='monotone', opacity = 0.5).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False, format="%B %Y")),
    alt.Y('lower_hpd_linear_50',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_linear_50' ),
    color=alt.Color('deme:N', scale=alt.Scale(domain=domain, range=range_) )
).properties(
    width=850,
    height=400
).transform_filter(
    (datum.upper_hpd_linear < 60) &(datum.upper_hpd_linear_50 < 10)
)

# band = alt.Chart(test).mark_area(
#     opacity=0.3, interpolate='monotone'
# ).encode(
#     alt.X('date:T', axis=alt.Axis(title="", grid=False)),
#     alt.Y('lower_hpd_linear'),#axis=None),#, scale=alt.Scale(domain=(0, 13))),
#     alt.Y2('upper_hpd_linear'),
#     color=alt.Color('deme:N', title = "Region", legend=alt.Legend(orient = "right",offset = -135, labelFontSize = 16, titleFontSize = 16, symbolSize = 110))
# ).properties(
#     width=850,
#     height=400
# ).transform_filter(
#     (datum.upper_hpd_linear < 20) &(datum.upper_hpd_linear_50 < 30)
# )
# band
# ne_plot = line + band
# (ne_plot).configure_axis(
#     labelFontSize=20,
#     titleFontSize=20
# )
line

alt.Chart(...)